# <h1 align="center"><font size="8">Applied Data Science Capstone</font></h1>
<h3 align="center"><font size="5">By Ian Riera Smolinska</font></h3>

<h2> Segmenting and Clustering Neighborhoods in Toronto. </h2>

<h3> Part 1: Scrapping of the Wikipedia list of postal codes of Canada and Dataframe creation. </h3>

Although this webpage is proposed https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M , due to continuous updates of the table that might cause some trouble with the scrapping, a fixed version of the webpage has been used: https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=933624196

In [1]:
# install the required libraries for scrapping
!pip install beautifulsoup4 # library for web scrapping
!pip install lxml # xml parser

In [2]:
# import the required libraries
import pandas as pd # library for data analsysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library for scrapping

print('Libraries imported.')

Libraries imported.


In [13]:
# get the webpage we want to analyze
source = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=933624196').text

# apply the scraper on the webpage
soup = BeautifulSoup(source, 'lxml')

# extract the table from the webpage
table = soup.find('table', {'class':'wikitable'})

# to get the data from the cells
table_data=""
for tr in table.find_all('tr'):
    row_data=""
    for tds in tr.find_all('td'):
        row_data=row_data+","+tds.text
    table_data=table_data+row_data[1:]
    
print(table_data[0:500])

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Queen's Park,Not assigned
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerso


In [14]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)
df

,PostalCode,Borough,Neighborhood


In [15]:
# get the table into rows
table_rows = table_data.split('\n')

# fill the dataframe with the information in each row
for row in table_rows:
    if row != '':
        data = row.split(',')
        postal_code = data[0]
        borough = data[1]
        neighborhood = data[2]
        df = df.append({'PostalCode' : postal_code, 'Borough' : borough, 'Neighborhood' : neighborhood}, ignore_index=True)

df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h4> Preprocessing </h4>


In [16]:
# remove the Postal Codes 'Not assigned' to a Borough
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [17]:
# assign the borough name to 'Not assigned' neighborhoods
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [18]:
# unify neighborhoods with same code
df = df.groupby(['PostalCode', 'Borough'], sort=True).agg( ', '.join)

# the index should be restored after deleting and merging rows from the dataframe
df = df.reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
df.shape

(103, 3)